In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install wikipedia

In [37]:
import wikipedia
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sentence_transformers import SentenceTransformer
result1 = wikipedia.search("Neural Networks")
result2 = wikipedia.search("Artificial Intelligence")
result3 = wikipedia.search("language")
results = result1+result2+result3

In [38]:
len(results)
print(result3)

['Language', 'Language (magazine)', 'Russian language', 'Assembly language', 'Language family', 'Bengali language', 'Programming language', 'Italian language', 'Languages of India', 'Portuguese language']


In [4]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [5]:
def create_cosine_matrix(final_content_list):
  matrix = []
  for para in final_content_list:
    embedding = model.encode(para)
    matrix.append(embedding)
    context_matrix = np.array(matrix)
    cos = cosine_similarity(context_matrix)
    
  for i in range(cos.shape[0]):
    for j in range(cos.shape[1]):
      if i == j:
        cos[i][j] = 0
  return cos

In [6]:
def get_pred_order(cos):
  final_order_lst = [0]
  for i in range(cos.shape[0]):
    sorted_tuples = []
    sorted_tuples = sorted(enumerate(cos[final_order_lst[-1]]), key = lambda i:i[1])
    # print(sorted_tuples)
    for elem in sorted_tuples[::-1]:
      if elem[0] not in final_order_lst:
        final_order_lst.append(elem[0])
  return final_order_lst

In [7]:
def get_pred_order_lookahead(cos):
  final_order_lst = [0]
  order_lst1 = [0]
  order_lst2 = [0]
  for i in range(cos.shape[0]):
    # sorted_list = []
    # sorted_list = cos[i].sort
    sorted_tuples = []
    # print(cos)
    sorted_tuples = sorted(enumerate(cos[final_order_lst[-1]]), key=lambda i: i[1])
    # print(sorted_tuples)
    for elem in sorted_tuples[::-1]:
      if elem[0] not in order_lst1:
        order_lst1.append(elem[0])
        prob1 = elem[1]
        break
      
    for elem in sorted_tuples[::-1]:
      if elem[0] not in order_lst2 and elem[0]!=order_lst1[-1]:
        order_lst2.append(elem[0])
        prob2 = elem[1]
        break
    
    sorted_tuples1 = sorted(enumerate(cos[order_lst1[-1]]), key = lambda i: i[1])
    sorted_tuples2 = sorted(enumerate(cos[order_lst2[-1]]), key = lambda i: i[1])

    for elem in sorted_tuples1[::-1]:
      if elem[0] not in order_lst1:
        order_lst1.append(elem[0])
        prob11 = elem[1]
        break

    for elem in sorted_tuples[::-1]:
      if elem[0] not in order_lst2 and elem[0]!=order_lst1[-1]:
        order_lst2.append(elem[0])
        prob22 = elem[1]
        break

    prob_1 = prob1 + prob11
    prob_2 = prob2 + prob22

    if prob_1 > prob_2:
      final_order_lst = order_lst1[:-1:]

    else:
      final_order_lst = order_lst2[:-1:]
    # print(order_lst)
    # break

  return (final_order_lst)


In [8]:
def accuracy(lst):
  total = 0
  correct = 0
  for i in range(len(lst)-1):
    if (lst[i+1] - lst[i] == 1):
      correct+=1
    total+=1
  return correct/total

In [43]:
def pred_accuracy(results):
  accuracy_lst = []
  for result in results:
    page = wikipedia.page(result)
    content = page.content
    content_list = content.split("\n")

    final_content_list = []
    for i in range (len(content_list)):
      if content_list[i] and len(content_list[i].split(" ")) > 30:
        final_content_list.append(content_list[i])

    if len(final_content_list) <= 1:
      continue

    cos = create_cosine_matrix(final_content_list)
    # print(cos.shape)
    final_order_lst = get_pred_order(cos)
    # print(final_order_lst)
    accuracy_lst.append(accuracy(final_order_lst))
    
    # print(final_order_lst)
  # print(accuracy_lst)
  return sum(accuracy_lst)/len(accuracy_lst)

In [ ]:
pred_accuracy(results)